In [1]:
from docx import Document
from llama_cpp import Llama

from copy import deepcopy
from smartdoc_utils import process_llm_output
from deterministic_preprocessor import DeterministicPreprocessor
from prompts import (
    karen_prompt,
    karen_system_prompt,
    llama_prompt,
    llama_system_prompt,
    gemma_prompt,
)
from llm_configs import karen_config, llama_config, gemma_config

# Initialize the model with GPU support
# llm = Llama(
#     model_path=settings.llm_model.model_name,
#     n_gpu_layers=-1,  # -1 means use all available GPU layers
#     n_ctx=16384,  # adjust based on your GPU memory
# )

# karen 7b q6_k creative
# llm = Llama.from_pretrained(
#     repo_id="FPHam/Karen_TheEditor_V2_CREATIVE_Mistral_7B-Q6_K-GGUF",
#     filename="karen_theeditor_v2_creative_mistral_7b.Q6_K.gguf",
#     n_gpu_layers=-1,
#     n_ctx=16384,
# )

# karen 7b q6_k strict
# llm = Llama.from_pretrained(
#     repo_id="TheBloke/Karen_TheEditor_V2_STRICT_Mistral_7B-GGUF",
#     filename="karen_theeditor_v2_strict_mistral_7b.Q6_K.gguf",
#     n_gpu_layers=-1,
#     n_ctx=16384,
# )

# llama 3.1 8b q6_k_l
# llm = Llama.from_pretrained(
#     repo_id="bartowski/Meta-Llama-3.1-8B-Instruct-GGUF",
#     filename="Meta-Llama-3.1-8B-Instruct-Q6_K_L.gguf",
#     n_gpu_layers=-1,
#     n_ctx=16384,
# )

# llama 3.2 3b fp16
# llm = Llama.from_pretrained(
#     repo_id="bartowski/Llama-3.2-3B-Instruct-GGUF",
#     filename="Llama-3.2-3B-Instruct-f16.gguf",
#     n_gpu_layers=-1,
#     n_ctx=16384,
# )

# gemma 2 9 b q6_k_l
# llm = Llama.from_pretrained(
#     repo_id="bartowski/gemma-2-9b-it-GGUF",
#     filename="gemma-2-9b-it-Q6_K_L.gguf",
#     n_gpu_layers=-1,
#     n_ctx=8192,
# )

# qwen2.5 7b q6_k_l
llm = Llama.from_pretrained(
    repo_id="bartowski/Qwen2.5-7B-Instruct-GGUF",
    filename="Qwen2.5-7B-Instruct-Q6_K_L.gguf",
    n_gpu_layers=-1,
    n_ctx=16384,
)

if "karen" in llm.model_path:
    generation_params = karen_config
    PROMPT_TEMPLATE = karen_prompt
    SYS_PROMPT = karen_system_prompt
elif "gemma" in llm.model_path:
    generation_params = gemma_config
    PROMPT_TEMPLATE = gemma_prompt
    SYS_PROMPT = None
else:
    generation_params = llama_config
    PROMPT_TEMPLATE = llama_prompt
    SYS_PROMPT = llama_system_prompt


def fetch_llm_response(
    text,
    prompt_template=PROMPT_TEMPLATE,
    sys_prompt=SYS_PROMPT,
):
    if sys_prompt is None:
        messages = [
            {
                "role": "user",
                "content": prompt_template.format(text=text),
            },
        ]
    else:
        messages = [
            {
                "role": "system",
                "content": sys_prompt,
            },
            {
                "role": "user",
                "content": prompt_template.format(text=text),
            },
        ]
    edited_text = llm.create_chat_completion(
        messages=messages,
        **generation_params,
    )
    response_text = edited_text["choices"][0]["message"]["content"].strip()
    print(f"PROMPT: \n {prompt_template.format(text=text)} \n\n")
    print(f"LLM RESPONSE: \n{response_text}\n\n")

    return response_text


# language checks, etc. before further processing
#
# Args:
#   input_doc: The original input document
#   corrections_doc: An empty document to populate with corrections
#
# Returns:
#   modified_doc: A modified version of the input document
#   corrections_doc: Populated with any corrections made during p
def pre_process_document(input_doc, corrections_doc):

    # logic for pre-processing Fonts, Australian Language Checks, etc.

    # -- Need to uncomment following 3 lines - the code has not been tested here yet!

    preprocessor = DeterministicPreprocessor()
    modified_doc, corrections_doc = preprocessor.pre_process_document(input_doc)

    return modified_doc, corrections_doc


def process_document_paragraphs(modified_doc, corrections_doc):

    # modified_doc = deepcopy(input_doc)
    # Process each paragraph
    for para in modified_doc.paragraphs:
        # Store the original formatting
        original_runs = para.runs.copy()

        if not para.text.strip():
            print("Skipping LLM CALL:")
            continue
        else:
            # Get the text content
            text = para.text

        llm_output_text = fetch_llm_response(text)

        # edits, corrections = document_postprocessing(llm_output_text)
        edits, corrections = process_llm_output(llm_output_text)
        print("EDITS : \n", edits)
        print("CORRECTIONS: \n", corrections)
        # Clear the paragraph and add the edited text
        para.clear()
        # para.add_run(edited_text['choices'][0]['text'].strip())
        para.add_run(edits)
        # Attempt to reapply formatting
        new_runs = para.runs
        for i, run in enumerate(new_runs):
            if i < len(original_runs):
                run.font.name = original_runs[i].font.name
                run.font.size = original_runs[i].font.size
                run.font.bold = original_runs[i].font.bold
                run.font.italic = original_runs[i].font.italic
                run.font.color.rgb = original_runs[i].font.color.rgb
                run.font.underline = original_runs[i].font.underline
                # Add more formatting attributes as needed

        # Let us log the corrections made
        corrections_doc.add_paragraph()
        corrections_doc.add_paragraph(f"Original Text : \n {text}")
        corrections_doc.add_paragraph(f"Edits : \n {edits}")
        corrections_doc.add_paragraph(f"Corrections: \n{llm_output_text}")

    return modified_doc, corrections_doc


def process_document_tables(modified_doc, corrections_doc):

    ## REPLACE this with logic of the modified doc
    # modified_doc = deepcopy(input_doc)
    # Iterate through all tables in the document
    for table in modified_doc.tables:
        print("IN Table")
        printed_cells = set()  # To keep track of cells that have been processed
        for r_index, row in enumerate(table.rows):
            for c_index, cell in enumerate(row.cells):
                cell_id = (r_index, c_index)  # Unique identifier for the cell

                # Skip this cell if it is already processed as part of a merged cell
                if cell_id in printed_cells:
                    continue

                # Detect merged cells
                is_merged = False
                for other_cell in row.cells:
                    if other_cell is not cell and other_cell._element is cell._element:
                        is_merged = True
                        break

                # If it's a merged cell, avoid processing duplicates
                if is_merged:
                    # Register this cell's element to skip duplicates
                    for merged_row_index, merged_row in enumerate(table.rows):
                        for merged_cell_index, merged_cell in enumerate(
                            merged_row.cells
                        ):
                            if merged_cell._element is cell._element:
                                printed_cells.add((merged_row_index, merged_cell_index))

                # Append '**' to the text of the cell if not already processed
                if cell.text.strip():  # Check if the cell is not empty
                    #                    cell.text += '*T*B*L'
                    for para in cell.paragraphs:
                        # Add an asterisk (*) to the end of each cell paragraph
                        print(para.text)
                        # Just a small check to see that we processed this
                        # para.add_run('*T')

                        # Store the original formatting
                        original_runs = para.runs.copy()
                        # let us call the llm
                        llm_output_text = fetch_llm_response(para.text)

                        # edits, corrections = document_postprocessing(llm_output_text)
                        edits, corrections = process_llm_output(llm_output_text)

                        # let us reapply formatting

                        # Clear the paragraph and add the edited text
                        para.clear()
                        # para.add_run(edited_text['choices'][0]['text'].strip())
                        para.add_run(edits)
                        # Attempt to reapply formatting
                        new_runs = para.runs
                        for i, run in enumerate(new_runs):
                            if i < len(original_runs):
                                run.font.name = original_runs[i].font.name
                                run.font.size = original_runs[i].font.size
                                run.font.bold = original_runs[i].font.bold
                                run.font.italic = original_runs[i].font.italic
                                run.font.color.rgb = original_runs[i].font.color.rgb
                                run.font.underline = original_runs[i].font.underline
                                # Add more formatting attributes as needed

                        # Let us log the corrections made
                        corrections_doc.add_paragraph()
                        corrections_doc.add_paragraph(f"Original Text : \n {para.text}")
                        corrections_doc.add_paragraph(
                            f"Corrections: \n{llm_output_text}"
                        )

            print()  # Newline after each row

    return modified_doc, corrections_doc


def proofread_and_correct_document(modified_doc, corrections_doc):
    corrections_doc.add_heading("Corrections Made", 0)

    # Correct using deterministic grammar checking
    modified_doc, corrections_doc = pre_process_document(modified_doc, corrections_doc)
    modified_doc, corrections_doc = process_document_paragraphs(
        modified_doc, corrections_doc
    )
    modified_doc, corrections_doc = process_document_tables(
        modified_doc, corrections_doc
    )

    return modified_doc, corrections_doc

/home/cdsw/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
llama_model_loader: loaded meta data with 38 key-value pairs and 339 tensors from /home/cdsw/.cache/huggingface/hub/models--bartowski--Qwen2.5-7B-Instruct-GGUF/snapshots/8911e8a47f92bac19d6f5c64a2e2095bd2f7d031/./Qwen2.5-7B-Instruct-Q6_K_L.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen2
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Qwen2.5 7B Instruct
llama_model_loader: - kv   3:                           general.finetu

In [9]:
input_path = "/home/cdsw/data/input.docx"
edit_path = "/home/cdsw/data/output_doc.docx"
correction_path = "/home/cdsw/data/correction_file.docx"
input_doc = Document(input_path)
modified_doc = deepcopy(input_doc)
# Create a document object for corrections
corrections_doc = Document()
modified_doc, corrections_doc = proofread_and_correct_document(
    modified_doc, corrections_doc
)
modified_doc.save(edit_path)
corrections_doc.save(correction_path)


input_doc = Document(input_path)
# make a deep copy because Redline removes the input document file
redline_input_doc = deepcopy(input_doc)
redline_input_doc_path="/home/cdsw/data/redline_input.docx"
redline_input_doc.save(redline_input_doc_path)

from python_redlines.engines import XmlPowerToolsEngine
wrapper = XmlPowerToolsEngine()

output_redlines = wrapper.run_redline('Smartdoc Processor', redline_input_doc_path, edit_path)
with open('/home/cdsw/data/redline_output.docx', 'wb') as f:
    f.write(output_redlines[0])

Skipping LLM CALL:
Skipping LLM CALL:
Skipping LLM CALL:
Skipping LLM CALL:


Llama.generate: 1268 prefix-match hit, remaining 19 prompt tokens to eval
llama_perf_context_print:        load time =   18472.61 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    19 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    28 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =  441594.36 ms /    47 tokens
Llama.generate: 1268 prefix-match hit, remaining 10 prompt tokens to eval


PROMPT: 
 Use the following examples as reference for editing.

------------------------------------------------------------
Example 1.
------------------------------------------------------------

Text: 
'I saw a    sample prodcut.  These product are below expeccted quality. I start to underdstand what he said is quite right. The correct valu is 12.5%% and 3rd postion. The time is 10:30am. The length of the object is 845mm and costs $1million. The sunise happens at 0750 EST and visibilty is 35 km. The right way too sumize and favor some advisor's is still to be found.'

1. Edited Text: 'I saw a sample product. These products are below expected quality. I started to understand what he said is quite right. The correct value is 12.5%% and 3rd position. The time is 10:30 am. The length of the object is 845mm and costs $1m. The sunrise happens at 0750 EST and visibility is 35 km. The right way to summarize and favor some advisors' is still to be found.'
2. Corrections:
   a) Removed extra 

llama_perf_context_print:        load time =   18472.61 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    10 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    17 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =  273642.42 ms /    27 tokens
Llama.generate: 1268 prefix-match hit, remaining 8 prompt tokens to eval


PROMPT: 
 Use the following examples as reference for editing.

------------------------------------------------------------
Example 1.
------------------------------------------------------------

Text: 
'I saw a    sample prodcut.  These product are below expeccted quality. I start to underdstand what he said is quite right. The correct valu is 12.5%% and 3rd postion. The time is 10:30am. The length of the object is 845mm and costs $1million. The sunise happens at 0750 EST and visibilty is 35 km. The right way too sumize and favor some advisor's is still to be found.'

1. Edited Text: 'I saw a sample product. These products are below expected quality. I started to understand what he said is quite right. The correct value is 12.5%% and 3rd position. The time is 10:30 am. The length of the object is 845mm and costs $1m. The sunrise happens at 0750 EST and visibility is 35 km. The right way to summarize and favor some advisors' is still to be found.'
2. Corrections:
   a) Removed extra 

...

llama_perf_context_print:        load time =   18472.61 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    17 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    22 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =  351330.92 ms /    39 tokens
Llama.generate: 1268 prefix-match hit, remaining 9 prompt tokens to eval


PROMPT: 
 Use the following examples as reference for editing.

------------------------------------------------------------
Example 1.
------------------------------------------------------------

Text: 
'I saw a    sample prodcut.  These product are below expeccted quality. I start to underdstand what he said is quite right. The correct valu is 12.5%% and 3rd postion. The time is 10:30am. The length of the object is 845mm and costs $1million. The sunise happens at 0750 EST and visibilty is 35 km. The right way too sumize and favor some advisor's is still to be found.'

1. Edited Text: 'I saw a sample product. These products are below expected quality. I started to understand what he said is quite right. The correct value is 12.5%% and 3rd position. The time is 10:30 am. The length of the object is 845mm and costs $1m. The sunrise happens at 0750 EST and visibility is 35 km. The right way to summarize and favor some advisors' is still to be found.'
2. Corrections:
   a) Removed extra 

llama_perf_context_print:        load time =   18472.61 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     9 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    15 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =  244666.07 ms /    24 tokens
Llama.generate: 1268 prefix-match hit, remaining 56 prompt tokens to eval


PROMPT: 
 Use the following examples as reference for editing.

------------------------------------------------------------
Example 1.
------------------------------------------------------------

Text: 
'I saw a    sample prodcut.  These product are below expeccted quality. I start to underdstand what he said is quite right. The correct valu is 12.5%% and 3rd postion. The time is 10:30am. The length of the object is 845mm and costs $1million. The sunise happens at 0750 EST and visibilty is 35 km. The right way too sumize and favor some advisor's is still to be found.'

1. Edited Text: 'I saw a sample product. These products are below expected quality. I started to understand what he said is quite right. The correct value is 12.5%% and 3rd position. The time is 10:30 am. The length of the object is 845mm and costs $1m. The sunrise happens at 0750 EST and visibility is 35 km. The right way to summarize and favor some advisors' is still to be found.'
2. Corrections:
   a) Removed extra 

KeyboardInterrupt: 